In [24]:
# Import package
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Specify url: url
res = []
url = 'https://www.boxofficemojo.com/alltime/domestic.htm?page='
urllist = []

#numofiter = 3
numofiter = 168


In [25]:
def urlAppend(urllist,url):
    return urllist.append(url) 
  
def genUrl(url,j):
    return url+str(j)+"&p=.htm"


In [26]:
def extractData(url, result):
    print("==EXTRACTING: " + url + " -- please wait")
    time.sleep(5)
    print("==EXTRACTING: " + url + " -- start")
    r = requests.get(url)
    html_doc = r.text
    
    # Write entire page locally for future processing
    f = open("../data_sample/data_raw/box_office_mojo/html/raw_" + url.replace('/', '_'), 'w')
    f.write(html_doc)  
    f.close()
    
    soup = BeautifulSoup(html_doc)
    for tr in soup.find_all('tr')[1:]:
        tds = tr.find_all('td')
        if str(tds[0]).lower().find('jump') != -1 or str(tds[0]).lower().find('specify date') != -1 or str(tds[0]).lower().find('lifetime gross') != -1:
            print("===SKIP===")
        else:
            if len(tds[0].find_all('table')) == 0:
                row = [trs.text.strip() for trs in tds if trs.text.strip()]
                if row:
                    if row[1] != "Title(click to view)":
                        title_href=tds[1].find('a')
                        if title_href:
                            row.append(title_href.get('href'))
                        else:
                            row.append('')
                        result.append(row)
    print("==EXTRACTING: " + url + " -- completed")
    return result

In [27]:
j=1
for i in range(1,numofiter):
    s=genUrl(url,j)
    urlAppend(urllist,s)
    j+=1
    

In [28]:
res1 = []
for url in urllist:
    res1 = extractData(url, res1)

df = pd.DataFrame(res1, columns=['rank','title','studio','lifetime_gross','year','href'])
print(len(df))  


==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=1&p=.htm -- please wait
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=1&p=.htm -- start
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=1&p=.htm -- completed
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=2&p=.htm -- please wait
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=2&p=.htm -- start
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=2&p=.htm -- completed
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=3&p=.htm -- please wait
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=3&p=.htm -- start
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=3&p=.htm -- completed
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=4&p=.htm -- please wait
==EXTRACTING: https://www.boxofficemojo.com/alltime/domestic.htm?page=4&p=.htm -- st

KeyboardInterrupt: 

In [69]:
import os
import psycopg2
print(df)

        rank                                 title studio lifetime_gross  \
0          1          Star Wars: The Force Awakens     BV   $936,662,225   
1          2                     Avengers: Endgame     BV   $858,373,000   
2          3                                Avatar    Fox   $760,507,625   
3          4                         Black Panther     BV   $700,059,566   
4          5                Avengers: Infinity War     BV   $678,815,482   
...      ...                                   ...    ...            ...   
16687  16688                           Dog Eat Dog    IFC            $80   
16688  16689                        Paranoid Girls    N/A            $78   
16689  16690  Confession of a Child of the Century  Cohen            $74   
16690  16691                            Storage 24  Magn.            $72   
16691  16692                           Zyzzyx Road   Reg.            $30   

        year                                            href  
0       2015            

In [67]:
def connect(db, user, password, host="localhost", port=5432):
    """
    Connects to the database and ensures there are tables.
    """
    conn = psycopg2.connect(**{
        "database": db,
        "user": user,
        "password": password,
        "host": host,
        "port": port

    })
    return conn

In [71]:
conn  = connect('db_name', 'user', 'pass', 'db_hose')

cursor = conn.cursor()
print("Query start: please wait")

cols = ",".join([str(i) for i in df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df.iterrows():
    print("Inserting row:"+str(i))
    sql = "INSERT INTO box_office_mojo_title (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    conn.commit()
    
print("Finish inserting rows:")
cursor.execute("SELECT count(id) FROM box_office_mojo_title")
box_office_mojo_title = cursor.fetchone()[0]
print("Query end:")
print("There are now %i box_office_mojo_title" % box_office_mojo_title)

conn.close()

Query start: please wait
Inserting row:0
Inserting row:1
Inserting row:2
Inserting row:3
Inserting row:4
Inserting row:5
Inserting row:6
Inserting row:7
Inserting row:8
Inserting row:9
Inserting row:10
Inserting row:11
Inserting row:12
Inserting row:13
Inserting row:14
Inserting row:15
Inserting row:16
Inserting row:17
Inserting row:18
Inserting row:19
Inserting row:20
Inserting row:21
Inserting row:22
Inserting row:23
Inserting row:24
Inserting row:25
Inserting row:26
Inserting row:27
Inserting row:28
Inserting row:29
Inserting row:30
Inserting row:31
Inserting row:32
Inserting row:33
Inserting row:34
Inserting row:35
Inserting row:36
Inserting row:37
Inserting row:38
Inserting row:39
Inserting row:40
Inserting row:41
Inserting row:42
Inserting row:43
Inserting row:44
Inserting row:45
Inserting row:46
Inserting row:47
Inserting row:48
Inserting row:49
Inserting row:50
Inserting row:51
Inserting row:52
Inserting row:53
Inserting row:54
Inserting row:55
Inserting row:56
Inserting row:57

Inserting row:460
Inserting row:461
Inserting row:462
Inserting row:463
Inserting row:464
Inserting row:465
Inserting row:466
Inserting row:467
Inserting row:468
Inserting row:469
Inserting row:470
Inserting row:471
Inserting row:472
Inserting row:473
Inserting row:474
Inserting row:475
Inserting row:476
Inserting row:477
Inserting row:478
Inserting row:479
Inserting row:480
Inserting row:481
Inserting row:482
Inserting row:483
Inserting row:484
Inserting row:485
Inserting row:486
Inserting row:487
Inserting row:488
Inserting row:489
Inserting row:490
Inserting row:491
Inserting row:492
Inserting row:493
Inserting row:494
Inserting row:495
Inserting row:496
Inserting row:497
Inserting row:498
Inserting row:499
Inserting row:500
Inserting row:501
Inserting row:502
Inserting row:503
Inserting row:504
Inserting row:505
Inserting row:506
Inserting row:507
Inserting row:508
Inserting row:509
Inserting row:510
Inserting row:511
Inserting row:512
Inserting row:513
Inserting row:514
Inserting 

Inserting row:916
Inserting row:917
Inserting row:918
Inserting row:919
Inserting row:920
Inserting row:921
Inserting row:922
Inserting row:923
Inserting row:924
Inserting row:925
Inserting row:926
Inserting row:927
Inserting row:928
Inserting row:929
Inserting row:930
Inserting row:931
Inserting row:932
Inserting row:933
Inserting row:934
Inserting row:935
Inserting row:936
Inserting row:937
Inserting row:938
Inserting row:939
Inserting row:940
Inserting row:941
Inserting row:942
Inserting row:943
Inserting row:944
Inserting row:945
Inserting row:946
Inserting row:947
Inserting row:948
Inserting row:949
Inserting row:950
Inserting row:951
Inserting row:952
Inserting row:953
Inserting row:954
Inserting row:955
Inserting row:956
Inserting row:957
Inserting row:958
Inserting row:959
Inserting row:960
Inserting row:961
Inserting row:962
Inserting row:963
Inserting row:964
Inserting row:965
Inserting row:966
Inserting row:967
Inserting row:968
Inserting row:969
Inserting row:970
Inserting 

Inserting row:1354
Inserting row:1355
Inserting row:1356
Inserting row:1357
Inserting row:1358
Inserting row:1359
Inserting row:1360
Inserting row:1361
Inserting row:1362
Inserting row:1363
Inserting row:1364
Inserting row:1365
Inserting row:1366
Inserting row:1367
Inserting row:1368
Inserting row:1369
Inserting row:1370
Inserting row:1371
Inserting row:1372
Inserting row:1373
Inserting row:1374
Inserting row:1375
Inserting row:1376
Inserting row:1377
Inserting row:1378
Inserting row:1379
Inserting row:1380
Inserting row:1381
Inserting row:1382
Inserting row:1383
Inserting row:1384
Inserting row:1385
Inserting row:1386
Inserting row:1387
Inserting row:1388
Inserting row:1389
Inserting row:1390
Inserting row:1391
Inserting row:1392
Inserting row:1393
Inserting row:1394
Inserting row:1395
Inserting row:1396
Inserting row:1397
Inserting row:1398
Inserting row:1399
Inserting row:1400
Inserting row:1401
Inserting row:1402
Inserting row:1403
Inserting row:1404
Inserting row:1405
Inserting ro

Inserting row:1786
Inserting row:1787
Inserting row:1788
Inserting row:1789
Inserting row:1790
Inserting row:1791
Inserting row:1792
Inserting row:1793
Inserting row:1794
Inserting row:1795
Inserting row:1796
Inserting row:1797
Inserting row:1798
Inserting row:1799
Inserting row:1800
Inserting row:1801
Inserting row:1802
Inserting row:1803
Inserting row:1804
Inserting row:1805
Inserting row:1806
Inserting row:1807
Inserting row:1808
Inserting row:1809
Inserting row:1810
Inserting row:1811
Inserting row:1812
Inserting row:1813
Inserting row:1814
Inserting row:1815
Inserting row:1816
Inserting row:1817
Inserting row:1818
Inserting row:1819
Inserting row:1820
Inserting row:1821
Inserting row:1822
Inserting row:1823
Inserting row:1824
Inserting row:1825
Inserting row:1826
Inserting row:1827
Inserting row:1828
Inserting row:1829
Inserting row:1830
Inserting row:1831
Inserting row:1832
Inserting row:1833
Inserting row:1834
Inserting row:1835
Inserting row:1836
Inserting row:1837
Inserting ro

...

Inserting row:14982
Inserting row:14983
Inserting row:14984
Inserting row:14985
Inserting row:14986
Inserting row:14987
Inserting row:14988
Inserting row:14989
Inserting row:14990
Inserting row:14991
Inserting row:14992
Inserting row:14993
Inserting row:14994
Inserting row:14995
Inserting row:14996
Inserting row:14997
Inserting row:14998
Inserting row:14999
Inserting row:15000
Inserting row:15001
Inserting row:15002
Inserting row:15003
Inserting row:15004
Inserting row:15005
Inserting row:15006
Inserting row:15007
Inserting row:15008
Inserting row:15009
Inserting row:15010
Inserting row:15011
Inserting row:15012
Inserting row:15013
Inserting row:15014
Inserting row:15015
Inserting row:15016
Inserting row:15017
Inserting row:15018
Inserting row:15019
Inserting row:15020
Inserting row:15021
Inserting row:15022
Inserting row:15023
Inserting row:15024
Inserting row:15025
Inserting row:15026
Inserting row:15027
Inserting row:15028
Inserting row:15029
Inserting row:15030
Inserting row:15031


Inserting row:15394
Inserting row:15395
Inserting row:15396
Inserting row:15397
Inserting row:15398
Inserting row:15399
Inserting row:15400
Inserting row:15401
Inserting row:15402
Inserting row:15403
Inserting row:15404
Inserting row:15405
Inserting row:15406
Inserting row:15407
Inserting row:15408
Inserting row:15409
Inserting row:15410
Inserting row:15411
Inserting row:15412
Inserting row:15413
Inserting row:15414
Inserting row:15415
Inserting row:15416
Inserting row:15417
Inserting row:15418
Inserting row:15419
Inserting row:15420
Inserting row:15421
Inserting row:15422
Inserting row:15423
Inserting row:15424
Inserting row:15425
Inserting row:15426
Inserting row:15427
Inserting row:15428
Inserting row:15429
Inserting row:15430
Inserting row:15431
Inserting row:15432
Inserting row:15433
Inserting row:15434
Inserting row:15435
Inserting row:15436
Inserting row:15437
Inserting row:15438
Inserting row:15439
Inserting row:15440
Inserting row:15441
Inserting row:15442
Inserting row:15443


Inserting row:15804
Inserting row:15805
Inserting row:15806
Inserting row:15807
Inserting row:15808
Inserting row:15809
Inserting row:15810
Inserting row:15811
Inserting row:15812
Inserting row:15813
Inserting row:15814
Inserting row:15815
Inserting row:15816
Inserting row:15817
Inserting row:15818
Inserting row:15819
Inserting row:15820
Inserting row:15821
Inserting row:15822
Inserting row:15823
Inserting row:15824
Inserting row:15825
Inserting row:15826
Inserting row:15827
Inserting row:15828
Inserting row:15829
Inserting row:15830
Inserting row:15831
Inserting row:15832
Inserting row:15833
Inserting row:15834
Inserting row:15835
Inserting row:15836
Inserting row:15837
Inserting row:15838
Inserting row:15839
Inserting row:15840
Inserting row:15841
Inserting row:15842
Inserting row:15843
Inserting row:15844
Inserting row:15845
Inserting row:15846
Inserting row:15847
Inserting row:15848
Inserting row:15849
Inserting row:15850
Inserting row:15851
Inserting row:15852
Inserting row:15853


Inserting row:16217
Inserting row:16218
Inserting row:16219
Inserting row:16220
Inserting row:16221
Inserting row:16222
Inserting row:16223
Inserting row:16224
Inserting row:16225
Inserting row:16226
Inserting row:16227
Inserting row:16228
Inserting row:16229
Inserting row:16230
Inserting row:16231
Inserting row:16232
Inserting row:16233
Inserting row:16234
Inserting row:16235
Inserting row:16236
Inserting row:16237
Inserting row:16238
Inserting row:16239
Inserting row:16240
Inserting row:16241
Inserting row:16242
Inserting row:16243
Inserting row:16244
Inserting row:16245
Inserting row:16246
Inserting row:16247
Inserting row:16248
Inserting row:16249
Inserting row:16250
Inserting row:16251
Inserting row:16252
Inserting row:16253
Inserting row:16254
Inserting row:16255
Inserting row:16256
Inserting row:16257
Inserting row:16258
Inserting row:16259
Inserting row:16260
Inserting row:16261
Inserting row:16262
Inserting row:16263
Inserting row:16264
Inserting row:16265
Inserting row:16266


Inserting row:16629
Inserting row:16630
Inserting row:16631
Inserting row:16632
Inserting row:16633
Inserting row:16634
Inserting row:16635
Inserting row:16636
Inserting row:16637
Inserting row:16638
Inserting row:16639
Inserting row:16640
Inserting row:16641
Inserting row:16642
Inserting row:16643
Inserting row:16644
Inserting row:16645
Inserting row:16646
Inserting row:16647
Inserting row:16648
Inserting row:16649
Inserting row:16650
Inserting row:16651
Inserting row:16652
Inserting row:16653
Inserting row:16654
Inserting row:16655
Inserting row:16656
Inserting row:16657
Inserting row:16658
Inserting row:16659
Inserting row:16660
Inserting row:16661
Inserting row:16662
Inserting row:16663
Inserting row:16664
Inserting row:16665
Inserting row:16666
Inserting row:16667
Inserting row:16668
Inserting row:16669
Inserting row:16670
Inserting row:16671
Inserting row:16672
Inserting row:16673
Inserting row:16674
Inserting row:16675
Inserting row:16676
Inserting row:16677
Inserting row:16678
